An AVL tree is a self-balancing binary search tree where the difference between the heights of left and right subtrees cannot be more than one for all nodes. Named after its inventors Adelson-Velsky and Landis, it ensures O(log n) time complexity for search, insertion, and deletion operations by maintaining this balance. This makes AVL trees efficient for dynamic sets and databases where frequent insertions and deletions occur.

In [1]:
class AVLNode:
    def __init__(self, key, height=1, left=None, right=None):
        self.key = key
        self.height = height
        self.left = left
        self.right = right

class AVLTree:
    def __init__(self):
        self.root = None

    def insert(self, key):
        if not self.root:
            self.root = AVLNode(key)
        else:
            self.root = self._insert(self.root, key)

    def _insert(self, node, key):
        if not node:
            return AVLNode(key)
        elif key < node.key:
            node.left = self._insert(node.left, key)
        else:
            node.right = self._insert(node.right, key)

        node.height = 1 + max(self._get_height(node.left), self._get_height(node.right))
        balance = self._get_balance(node)

        if balance > 1 and key < node.left.key:
            return self._right_rotate(node)
        if balance < -1 and key > node.right.key:
            return self._left_rotate(node)
        if balance > 1 and key > node.left.key:
            node.left = self._left_rotate(node.left)
            return self._right_rotate(node)
        if balance < -1 and key < node.right.key:
            node.right = self._right_rotate(node.right)
            return self._left_rotate(node)

        return node

    def _left_rotate(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self._get_height(z.left), self._get_height(z.right))
        y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))
        return y

    def _right_rotate(self, z):
        y = z.left
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self._get_height(z.left), self._get_height(z.right))
        y.height = 1 + max(self._get_height(y.left), self._get_height(y.right))
        return y

    def _get_height(self, node):
        if not node:
            return 0
        return node.height

    def _get_balance(self, node):
        if not node:
            return 0
        return self._get_height(node.left) - self._get_height(node.right)

    def pre_order(self, node):
        if not node:
            return
        print("{0} ".format(node.key), end="")
        self.pre_order(node.left)
        self.pre_order(node.right)

# Example usage:
avl = AVLTree()
keys = [10, 20, 30, 40, 50, 25]
for key in keys:
    avl.insert(key)

print("Preorder traversal of the constructed AVL tree is:")
avl.pre_order(avl.root)

Preorder traversal of the constructed AVL tree is:
30 20 10 25 40 50 

In [2]:
class Task:
    def __init__(self, priority, description):
        self.priority = priority
        self.description = description

    def __lt__(self, other):
        return self.priority < other.priority

    def __str__(self):
        return f"Task(priority={self.priority}, description='{self.description}')"

class TaskScheduler(AVLTree):
    def insert_task(self, priority, description):
        task = Task(priority, description)
        self.insert(task)

    def get_highest_priority_task(self):
        if not self.root:
            return None
        return self._get_min_value_node(self.root).key

    def remove_highest_priority_task(self):
        if not self.root:
            return None
        min_node = self._get_min_value_node(self.root)
        self.root = self._delete_node(self.root, min_node.key)
        return min_node.key

    def _get_min_value_node(self, node):
        if node is None or node.left is None:
            return node
        return self._get_min_value_node(node.left)

    def _delete_node(self, root, key):
        if not root:
            return root
        elif key < root.key:
            root.left = self._delete_node(root.left, key)
        elif key > root.key:
            root.right = self._delete_node(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            temp = self._get_min_value_node(root.right)
            root.key = temp.key
            root.right = self._delete_node(root.right, temp.key)
        root.height = 1 + max(self._get_height(root.left), self._get_height(root.right))
        balance = self._get_balance(root)
        if balance > 1 and self._get_balance(root.left) >= 0:
            return self._right_rotate(root)
        if balance > 1 and self._get_balance(root.left) < 0:
            root.left = self._left_rotate(root.left)
            return self._right_rotate(root)
        if balance < -1 and self._get_balance(root.right) <= 0:
            return self._left_rotate(root)
        if balance < -1 and self._get_balance(root.right) > 0:
            root.right = self._right_rotate(root.right)
            return self._left_rotate(root)
        return root

# Example usage:
scheduler = TaskScheduler()
tasks = [(3, "Task 1"), (1, "Task 2"), (2, "Task 3"), (5, "Task 4")]

for priority, description in tasks:
    scheduler.insert_task(priority, description)

print("Highest priority task:", scheduler.get_highest_priority_task())
scheduler.remove_highest_priority_task()
print("Highest priority task after removal:", scheduler.get_highest_priority_task())

Highest priority task: Task(priority=1, description='Task 2')
Highest priority task after removal: Task(priority=2, description='Task 3')


In [3]:
class Event:
    def __init__(self, time, description):
        self.time = time
        self.description = description

    def __lt__(self, other):
        return self.time < other.time

    def __str__(self):
        return f"Event(time={self.time}, description='{self.description}')"

class Calendar(AVLTree):
    def add_event(self, time, description):
        event = Event(time, description)
        self.insert(event)

    def remove_event(self, time):
        event = Event(time, "")
        self.root = self._delete_node(self.root, event)

    def get_next_event(self):
        if not self.root:
            return None
        return self._get_min_value_node(self.root).key

    def _get_min_value_node(self, node):
        if node is None or node.left is None:
            return node
        return self._get_min_value_node(node.left)

    def _delete_node(self, root, key):
        if not root:
            return root
        elif key < root.key:
            root.left = self._delete_node(root.left, key)
        elif key > root.key:
            root.right = self._delete_node(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            temp = self._get_min_value_node(root.right)
            root.key = temp.key
            root.right = self._delete_node(root.right, temp.key)
        root.height = 1 + max(self._get_height(root.left), self._get_height(root.right))
        balance = self._get_balance(root)
        if balance > 1 and self._get_balance(root.left) >= 0:
            return self._right_rotate(root)
        if balance > 1 and self._get_balance(root.left) < 0:
            root.left = self._left_rotate(root.left)
            return self._right_rotate(root)
        if balance < -1 and self._get_balance(root.right) <= 0:
            return self._left_rotate(root)
        if balance < -1 and self._get_balance(root.right) > 0:
            root.right = self._right_rotate(root.right)
            return self._left_rotate(root)
        return root

# Example usage:
calendar = Calendar()
events = [(9, "Breakfast"), (13, "Lunch"), (18, "Dinner"), (8, "Morning Exercise")]

for time, description in events:
    calendar.add_event(time, description)

print("Next event:", calendar.get_next_event())
calendar.remove_event(8)
print("Next event after removal:", calendar.get_next_event())

Next event: Event(time=8, description='Morning Exercise')
Next event after removal: Event(time=9, description='Breakfast')
